<a href="https://colab.research.google.com/github/JD-xd/sentiment-analysis-using-NLP-/blob/main/nlp_sentiment_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import all imp librierys
import nltk
import pandas as pd
import numpy as np
import re
import string
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer


In [ ]:
# punt , stopwords , wordnet
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#load the data
df = pd.read_csv('/content/sentiment_dataset_raw.csv')
df

,Review,sentiment
0,I absolutely loved this movie! 😊 😍,positive
1,"Terrible experience, would not recommend. !!",negative
2,I absolutely loved this movie! 😊 😍,positive
3,"Had a great time with friends, everything was ...",positive
4,I absolutely loved this movie! 😊,positive
...,...,...
5995,What a fantastic experience. Will come again! !!!,positive
5996,Worst product ever. Total waste of money!,negative
5997,"Had a great time with friends, everything was ...",positive
5998,"The service was excellent, highly recommend!",positive


In [ ]:
# lower caseing
# convert all sentence word into lower case
df['Review'] = df['Review'].str.lower()
df

,Review,sentiment
0,i absolutely loved this movie! 😊 😍,positive
1,"terrible experience, would not recommend. !!",negative
2,i absolutely loved this movie! 😊 😍,positive
3,"had a great time with friends, everything was ...",positive
4,i absolutely loved this movie! 😊,positive
...,...,...
5995,what a fantastic experience. will come again! !!!,positive
5996,worst product ever. total waste of money!,negative
5997,"had a great time with friends, everything was ...",positive
5998,"the service was excellent, highly recommend!",positive


In [ ]:
# remove html tags
df['Review'] = df['Review'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
df

,Review,sentiment
0,i absolutely loved this movie! 😊 😍,positive
1,"terrible experience, would not recommend. !!",negative
2,i absolutely loved this movie! 😊 😍,positive
3,"had a great time with friends, everything was ...",positive
4,i absolutely loved this movie! 😊,positive
...,...,...
5995,what a fantastic experience. will come again! !!!,positive
5996,worst product ever. total waste of money!,negative
5997,"had a great time with friends, everything was ...",positive
5998,"the service was excellent, highly recommend!",positive


In [ ]:
df[df['Review'].str.contains('<')]

,Review,sentiment


In [ ]:
# checking and removing  url's
df[df['Review'].str.contains('http')]  # there are http contant

,Review,sentiment
5,"the service was excellent, highly recommend! h...",positive
14,"superb product quality, very satisfied. http:/...",positive
24,i hated every minute of it. 😡 http://example.com,negative
41,"superb product quality, very satisfied. http:/...",positive
50,not worth the time or the price. http://exampl...,negative
...,...,...
5959,i hated every minute of it. 😡 http://example.com,negative
5978,"had a great time with friends, everything was ...",positive
5980,i'm very disappointed with the service. http:/...,negative
5981,i hated every minute of it. 😡 http://example.com,negative


In [ ]:
df['Review'] = df['Review'].apply(lambda x: re.sub(r'http\S+', '', x))

In [ ]:
df[df['Review'].str.contains('http')]

,Review,sentiment


In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
z=["%$T#Q&T*lsdvnsvlsdvn."]
z= lambda x: x.translate(str.maketrans('', '', string.punctuation))

In [ ]:
z

<function __main__.<lambda>(x)>

In [ ]:
df['Review'] =df['Review'].apply (lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [ ]:
df['Review']

,Review
0,i absolutely loved this movie 😊 😍
1,terrible experience would not recommend
2,i absolutely loved this movie 😊 😍
3,had a great time with friends everything was p...
4,i absolutely loved this movie 😊
...,...
5995,what a fantastic experience will come again
5996,worst product ever total waste of money
5997,had a great time with friends everything was p...
5998,the service was excellent highly recommend


In [ ]:
# chat word treatement --- u --- you
# convert informal txt to in standard txt
chat_word_map={
    'u' : 'you',
    'ur' : 'your',
    'lol' : 'laugh out loud',
    'omg' : 'oh my god',
    'idk' : 'i do not know',
    'btw' : 'by the way',
    'brb' : 'be right back'
}



In [ ]:
def expand_chat_word(text):
  return ' '.join([chat_word_map[word.lower()] if word.lower() in chat_word_map else word for word in text.split()])

In [ ]:
df['Review'] = df['Review'].apply(expand_chat_word)
df

,Review,sentiment
0,i absolutely loved this movie 😊 😍,positive
1,terrible experience would not recommend,negative
2,i absolutely loved this movie 😊 😍,positive
3,had a great time with friends everything was p...,positive
4,i absolutely loved this movie 😊,positive
...,...,...
5995,what a fantastic experience will come again,positive
5996,worst product ever total waste of money,negative
5997,had a great time with friends everything was p...,positive
5998,the service was excellent highly recommend,positive


In [ ]:
#spelling correction (optional but slow)
# correct miss spelled word
# use tool >> textblob,symspell or autocorrect

from textblob import TextBlob
df['Review'].apply(lambda x: str(TextBlob(x).correct()))


,Review
0,i absolutely loved this movie 😊 😍
1,terrible experience would not recommend
2,i absolutely loved this movie 😊 😍
3,had a great time with friends everything was p...
4,i absolutely loved this movie 😊
...,...
5995,what a fantastic experience will come again
5996,worst product ever total waste of money
5997,had a great time with friends everything was p...
5998,the service was excellent highly recommend


In [ ]:
# remove stopwords
# remove comman word like ' the is am are we and or not '
stop_words = set(stopwords.words('english'))
print (stop_words)

{"it's", "needn't", 'had', 'both', 'themselves', 'few', 've', "wasn't", 'through', 'so', 're', 'at', 'itself', "mightn't", 'd', 'that', "they've", 'while', 'most', 'aren', "we'll", "shan't", 'mustn', "hadn't", 'yourself', 'couldn', 'do', "i'd", 'until', 'why', "don't", 'during', 'with', 'have', 'her', 'weren', 'theirs', 'any', 'too', 'hasn', 'for', "it'd", "we've", 'down', 'above', 'i', 'of', "won't", 'their', 'who', 'they', 'you', "should've", 'himself', 'below', 'after', 'were', 'she', 'then', 'shan', "didn't", 'very', 'when', 'just', "he's", 'before', 'its', "they're", "she's", "haven't", 'being', "he'd", "she'd", 'ma', 'between', 'hadn', "i've", 'we', 'ourselves', 'will', 'my', 'here', "it'll", 'should', 'don', 'some', "aren't", 'not', "that'll", 'this', 'how', 'hers', 'only', 'same', 'those', 'him', 'yourselves', 'won', "they'd", "she'll", 'doesn', 'own', 'yours', 'where', 'a', 'in', 'now', "we're", 'and', 'the', 't', 'am', 'an', 'each', 'm', "they'll", 'doing', 'been', 'it', 'our

In [ ]:
from pygments import token
from nltk.sem.drt import Tokens
# tokeniztion
from nltk.tokenize import word_tokenize
def remove_stop_words(text):
  tokens=word_tokenize(text)
  return ' '.join([word for word in tokens if word not in stop_words])

In [ ]:
import nltk
nltk.download('punkt_tab')
df['Review'] = df['Review'].apply(remove_stop_words)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# handling imoji's
# convert emotion images into text

!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 23.8 MB/s eta 0:00:00


In [ ]:
import emoji
emoji.demojize('☠️☠️')

':skull_and_crossbones::skull_and_crossbones:'

In [ ]:
df['Review']=df['Review'].apply(lambda x: emoji.demojize(x,delimiters=(' ',' ')))

In [ ]:
# tokenization
# split sentence into words

df['Tokenize Review'] = df['Review'].apply(word_tokenize)
df

,Review,sentiment,Tokenize Review
0,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s..."
1,terrible experience would recommend,negative,"[terrible, experience, would, recommend]"
2,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s..."
3,great time friends everything perfect laugh loud,positive,"[great, time, friends, everything, perfect, la..."
4,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s..."
...,...,...,...
5995,fantastic experience come,positive,"[fantastic, experience, come]"
5996,worst product ever total waste money,negative,"[worst, product, ever, total, waste, money]"
5997,great time friends everything perfect,positive,"[great, time, friends, everything, perfect]"
5998,service excellent highly recommend,positive,"[service, excellent, highly, recommend]"


In [ ]:
# steming : reduce word to base / root words (eg. play -- playing --- playful)

stemmer=PorterStemmer()
def steam_word(text):
  return [stemmer.stem(word) for word in text]
df['clean_Review']=df['Tokenize Review'].apply(steam_word)
df

,Review,sentiment,Tokenize Review,clean_Review
0,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s...","[absolut, love, movi, smiling_face_with_smilin..."
1,terrible experience would recommend,negative,"[terrible, experience, would, recommend]","[terribl, experi, would, recommend]"
2,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s...","[absolut, love, movi, smiling_face_with_smilin..."
3,great time friends everything perfect laugh loud,positive,"[great, time, friends, everything, perfect, la...","[great, time, friend, everyth, perfect, laugh,..."
4,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s...","[absolut, love, movi, smiling_face_with_smilin..."
...,...,...,...,...
5995,fantastic experience come,positive,"[fantastic, experience, come]","[fantast, experi, come]"
5996,worst product ever total waste money,negative,"[worst, product, ever, total, waste, money]","[worst, product, ever, total, wast, money]"
5997,great time friends everything perfect,positive,"[great, time, friends, everything, perfect]","[great, time, friend, everyth, perfect]"
5998,service excellent highly recommend,positive,"[service, excellent, highly, recommend]","[servic, excel, highli, recommend]"


In [ ]:
# limitization : convert words to dict from (eg. better -- good )

lemmatizer=WordNetLemmatizer()
def lemmatizer_word(text):
    return [lemmatizer.lemmatize(word) for word in text]
df['clean_Review']=df['clean_Review'].apply(lemmatizer_word)
df

,Review,sentiment,Tokenize Review,clean_Review
0,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s...","[absolut, love, movi, smiling_face_with_smilin..."
1,terrible experience would recommend,negative,"[terrible, experience, would, recommend]","[terribl, experi, would, recommend]"
2,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s...","[absolut, love, movi, smiling_face_with_smilin..."
3,great time friends everything perfect laugh loud,positive,"[great, time, friends, everything, perfect, la...","[great, time, friend, everyth, perfect, laugh,..."
4,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s...","[absolut, love, movi, smiling_face_with_smilin..."
...,...,...,...,...
5995,fantastic experience come,positive,"[fantastic, experience, come]","[fantast, experi, come]"
5996,worst product ever total waste money,negative,"[worst, product, ever, total, waste, money]","[worst, product, ever, total, wast, money]"
5997,great time friends everything perfect,positive,"[great, time, friends, everything, perfect]","[great, time, friend, everyth, perfect]"
5998,service excellent highly recommend,positive,"[service, excellent, highly, recommend]","[servic, excel, highli, recommend]"


In [ ]:
# bow : vextorization
from sklearn.feature_extraction.text import CountVectorizer
df['string_review']=df['clean_Review'].apply(lambda x: ' '.join(x))
vectorizer=CountVectorizer()
bow=vectorizer.fit_transform(df['string_review'])
print(bow)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 27457 stored elements and shape (6000, 39)>
  Coords	Values
  (0, 0)	1
  (0, 18)	1
  (0, 21)	1
  (0, 30)	1
  (0, 29)	1
  (0, 9)	1
  (1, 32)	1
  (1, 8)	1
  (1, 38)	1
  (1, 26)	1
  (2, 0)	1
  (2, 18)	1
  (2, 21)	1
  (2, 30)	1
  (2, 29)	1
  (2, 9)	1
  (3, 12)	1
  (3, 33)	1
  (3, 11)	1
  (3, 6)	1
  (3, 22)	1
  (3, 16)	1
  (3, 17)	1
  (4, 0)	1
  (4, 18)	1
  :	:
  (5994, 1)	1
  (5995, 8)	1
  (5995, 10)	1
  (5995, 1)	1
  (5996, 24)	1
  (5996, 36)	1
  (5996, 4)	1
  (5996, 34)	1
  (5996, 35)	1
  (5996, 20)	1
  (5997, 12)	1
  (5997, 33)	1
  (5997, 11)	1
  (5997, 6)	1
  (5997, 22)	1
  (5998, 26)	1
  (5998, 28)	1
  (5998, 7)	1
  (5998, 14)	1
  (5999, 24)	1
  (5999, 36)	1
  (5999, 4)	1
  (5999, 34)	1
  (5999, 35)	1
  (5999, 20)	1


In [ ]:
vectorizer.vocabulary_
bow[0].toarray()

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(ngram_range=(1,2))
n_gram=vectorizer.fit_transform(df['string_review'])
n_gram[0].toarray()

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0]])

In [ ]:
n_gram.shape

(6000, 89)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
x_tfidf=vectorizer.fit_transform(df['string_review'])

In [ ]:
vectorizer


TfidfVectorizer()

In [ ]:
x_tfidf[0].toarray()

array([[0.42537028, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.37164533,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.42537028, 0.        ,
        0.        , 0.42537028, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.37164533,
        0.42537028, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]])

In [ ]:
from scipy.sparse import csr_matrix
x = x_tfidf
x

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 27457 stored elements and shape (6000, 39)>

In [ ]:
x[0].toarray()

array([[0.42537028, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.37164533,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.42537028, 0.        ,
        0.        , 0.42537028, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.37164533,
        0.42537028, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]])

In [ ]:
# store each row as a dence numpy array in a list

feature_list = [x[i].toarray().flatten() for i in range (x.shape[0])]
feature_list

[array([0.42537028, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.37164533,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.42537028, 0.        ,
        0.        , 0.42537028, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.37164533,
        0.42537028, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.44215901, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.43667923, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.55398857, 0.        , 0.

In [ ]:
# or if you want to store as spaece vector (for storage efficency )
sparce_feature_list=[x[i] for i in range (x.shape[0])]
sparce_feature_list

[<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 6 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 4 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 6 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 7 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 4 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 4 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 5 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 6 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 4 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 4 stored elements 

In [ ]:
type(sparce_feature_list)

list

In [ ]:
from scipy.sparse import save_npz   # NPZ stands for NumPy Zipped, a file format for storing multiple NumPy arrays (like data tables or tensors in Python) into a single, compressed file
save_npz('tfidf_Matrix.npz',x)   # why x -- x is csr_matrix

In [ ]:
from scipy.sparse import load_npz
x=load_npz('tfidf_Matrix.npz')

In [ ]:
x

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 27457 stored elements and shape (6000, 39)>

In [ ]:
y = df['sentiment']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
y

array([1, 0, 1, ..., 1, 1, 0])

In [ ]:
# import all imp libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [ ]:
x_array = x.toarray()  # convert tfidf sparse matrix to array


In [ ]:
# train test split
x_train,x_test,y_train,y_test=train_test_split(x_array,y,test_size=0.2,random_state=42)


In [ ]:
# train the model
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [ ]:
# pred
y_pred = model.predict(x_test)
y_pred

array([1, 0, 1, ..., 1, 0, 1])

In [ ]:
accuracy_score(y_test,y_pred)

1.0

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       571
           1       1.00      1.00      1.00       629

    accuracy                           1.00      1200
   macro avg       1.00      1.00      1.00      1200
weighted avg       1.00      1.00      1.00      1200

